# multi_agent/functions_agents.py

## Dependencies

In [2]:
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.prebuilt import ToolExecutor

import functions_python

## detect_trend

In [ ]:
class PropertiesCalculateTrend(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	start_datetime: Optional[str] = Field(None, description="The start timestamp of period over which the analysis is done. \
The format of the date should be in the following format %b-%d-%y %H:%M:%S like this example: May-1-2024 13:27:49")
	end_datetime: Optional[str] = Field(None, description="The end timestamp of period over which the analysis is done. \
The format of the date should be in the following format %b-%d-%y %H:%M:%S like this example: May-1-2024 13:27:49. \
The user can set this parameter to now. In this situation this parameter's value is the current date time.")

	lookback: Optional[str] = Field(None, description="The number of seconds, minutes, hours, days, weeks, months or years to look back for calculating the trend of the given symbol. \
This parameter determines the depth of historical data to be considered in the analysis. The format of this value must obey one of the following examples: 30 seconds, 10 minutes, 2 hours, 5 days, 3 weeks, 2 months and 3 years. \
Either start_datetime along with end_datetime should be specified or lookback should be specified but both cases should not happen simultaneously.")

In [ ]:
class CalculateTrend(BaseTool):
	name = "detect_trend"
	description = """Analyzes the trend of a specified financial instrument over a given time range. \
It is designed primarily for financial data analysis, enabling users to gauge the general direction of a security or market index. \
Whether start_datetime with end_datetime, end_datetime with lookback or lookback parameters could be valued for determining the period over which's trend wants to be detected. \
The function returns a numerical value that indicates the trend intensity and direction within the specified parameters. \
Returns a number between -3 and 3 that represents the trend’s intensity and direction. The value is interpreted as follows: \
\n -3: strong bearish (downward) trend \
\n -2: moderate bearish (downward) trend \
\n -1: mild bearish (downward) trend \
\n 0: without significant trend \
\n 3: strong bullish (upward) trend \
\n 2: moderate bullish (upward) trend \
\n 1: mild bullish (upward) trend"""

	args_schema: Type[BaseModel] = PropertiesCalculateTrend
	return_direct: bool = True

	def _run(
			self, symbol: str = None, start_datetime: str = None, end_datetime: str = None, lookback: str = None
    ) -> dict:

		function_arguments = {
						"symbol": symbol,
						"start_datetime": start_datetime,
						"end_datetime": end_datetime,
						"lookback": lookback
						}
		
		fc = functions_python.FunctionCalls()

		# if state
		return fc.detect_trend(function_arguments)

## calculate_sr

In [ ]:
class PropertiesCalculateSR(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")
	timeframe: Optional[str] = Field(None, description="Specifies the timeframe of the candlestick chart to be analyzed. \
This parameter defines the granularity of the data used for calculating the levels. The only allowed formats would like 3h, 20min, 1d.")
	lookback_days: Optional[str] = Field(None, description="The number of days to look back for calculating the support and resistance levels. \
This parameter determines the depth of historical data to be considered in the analysis. (e.g. 10 days)")

In [ ]:
class CalculateSR(BaseTool):
	name = "calculate_sr"
	description = """Support and resistance levels represent price points on a chart where the odds favor a pause or reversal of a prevailing trend. \
This function analyzes candlestick charts over a specified timeframe and lookback period to calculate these levels and their respective strengths. \
Returns a dictionary containing five lists, each corresponding to a specific aspect of the calculated support and resistance levels: \
1. levels_prices (list of floats): The prices at which support and resistance levels have been identified. \
2. levels_start_timestamps (list of timestamps) \
3. levels_detect_timestamps (list of timestamps) \
4. levels_end_timestamps (list of timestamps) \
5. levels_scores (list of floats): Scores associated with each level, indicating the strength or significance of the level. Higher scores typically imply stronger levels."""

	args_schema: Type[BaseModel] = PropertiesCalculateSR

	def _run(
			self, symbol: str = None, timeframe: str = None, lookback_days: str = None
    ) -> dict:
		parameters = {
						"symbol": symbol,
						"timeframe": timeframe,
						"lookback_days": lookback_days
						}
		
		fc = functions_python.FunctionCalls()

		return fc.calculate_sr(parameters)

## calculate_sl

In [ ]:
class PropertiesCalculateSl(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	method: Optional[str] = Field(None, description="shows the method of SL calculation.")

	direction: Optional[int] = Field(None, description="-1: means the user want to calculate stoploss for a short position. 1: means the user want to calculate stoploss for a long position")

	lookback: Optional[int] = Field(None, description="it is used when the method is set to 'minmax' and shows the number of candles that the SL is calculated based on them.")

	neighborhood: Optional[int] = Field(None, description="A parameter that is used in the swing method to define the range or window within which swings are detected. example: \
If the 'neighborhood' parameter is set to 3, it means that the swing detection is based on considering 3 candles to the \
left and 3 candles to the right of the swing point.")

	atr_coef: Optional[int] = Field(None, description="it is used if the method is 'atr' and shows the coefficient of atr")

In [ ]:
class CalculateSL(BaseTool):
	name = "calculate_sl"
	description = """Stoploss (SL) is a limitation for potential losses in a position. It's below the current price for long position and above it for short position. \
Distance between the SL and current price is named risk value. This function calculates the SL based o some different methods. \
Returns A dictionary same as this: \
{'sl': [17542.5], 'risk': [268.5], 'info': ['calculated based on maximum high price of previous 100 candles']} \
which includes sl value, risk on the trade and an information. \
If user don't select any method for sl calculation or select "level" method, or zigzag method the otput can include \
more than one stoploss and the values type in the output can be a list such as this \
{'sl': [17542.5, 17818.25, 17858.5, 17882.5, 18518.75], 'risk': [268.5, 7.25, 47.5, 71.5, 707.75], 'info': ['minmax', 'swing', 'atr', '5min_SR', 'daily_SR']} \
It includes a list of stoplosses and the risk on them and finally the level or method name of stoploss."""

	args_schema: Type[BaseModel] = PropertiesCalculateSl

	def _run(
        self, symbol: str = None, method: str = None, direction: int = None,
		lookback: int = None, neighborhood: int = None, atr_coef: int = None
    ) -> int:
		parameters = {
                      "symbol": symbol,
                      "method": method,
                      "direction": direction,
                      "lookback": lookback,
                      "neighborhood": neighborhood,
                      "atr_coef": atr_coef
                      }
		
		fc = functions_python.FunctionCalls()

		return fc.calculate_sl(parameters)

## calculate_tp

In [ ]:
class PropertiesCalculateTp(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	direction: Optional[int] = Field(None, description="-1: means the user want to calculate stoploss for a short position. 1: means the user want to calculate stoploss for a long position")

	stoploss: Optional[int] = Field(None, description="the value for stoploss")

In [ ]:
class CalculateTp(BaseTool):
	name = "calculate_tp"
	description = """Take profit (TP) is opposite of the stop-loss (SL) and is based on maximum reward that we intend to achieve from a trade. \
It represents the price level at which a trader aims to close a position to secure profits before the market reverses. \
Returns list of price for take-profit and information for each price For exampe: \
{'tp': [5139.25, 5140.25, 5144.0], 'info': ['calculated based on the level VWAP_Top_Band_2', 'calculated based on the level Overnight_high', 'calculated based on the level VWAP_Top_Band_3']}"""

	args_schema: Type[BaseModel] = PropertiesCalculateTp

	def _run(
        self, symbol: str = None, direction: int = None, stoploss: int = None
    ) -> int:
		parameters = {
						"symbol": symbol,
						"direction": direction,
						"stoploss": stoploss
						}
		
		fc = functions_python.FunctionCalls()

		return fc.calculate_tp(parameters)

## get_bias

In [ ]:
class PropertiesBiasDetection(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	method: Optional[str] = Field(None, description="The user can choose from different methods including MC, Zigzag trend, \
Trend detection, weekly wvap, candle stick pattern, cross ma, vp detection ,power & counter ratio.")

In [ ]:
class BiasDetection(BaseTool):
	name = "bias_detection"
	description = """Detecting trading bias through different methods or Detecting the appropriate entry point for a long or short trade.
Returns a number between -3 and 3 that represents the trend’s intensity and direction. The value is interpreted as follows:
-3: Strong downward , -2: downward -1: Weak downward, 0: No significant trend / Neutral, 1: Weak upward, 2.upward, 3: Strong upward"""

	args_schema: Type[BaseModel] = PropertiesBiasDetection

	def _run(
        self, symbol: str = None, method: str = None
    ) -> dict:

		parameters = {
						"symbol": symbol,
						"method": method
						}
		
		fc = functions_python.FunctionCalls()

		return fc.get_bias(parameters)

## irrelevant_handler

In [1]:
class Handlers:
	def __init__(self, client, ChatWithOpenai):
		self.client = client
		self.default_message = ["Check the following text for greeting words and do as the system message said."]
		self.ChatWithOpenai = ChatWithOpenai

	def handler_zero(self):
		handler_zero_openai = self.ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input Not answer it.\
If the user input contains greeting words like ‘hello’, ‘hi’, and so on, \
then you should remove these words. \
Note that the final response is either the input with the greeting word removed, \
or ‘None’ if the input consists only of a greeting word. \
if there is no greeting word in the input, the response is the last user input itself \
without any changes. \
No other responses are possible. \
Here are some examples and the valid responses: \
Example 1 (when there is no greeting word): \
{ input: ‘What is the weather?’ response: ‘What is the weather?’ } \
Example 2 (when there is a greeting word): \
{ input: ‘Hi can you speak French?’ response: ‘Can you speak French?’ } \
Example 3 (when there is just one or more greeting words): \
{ input: ‘Hello.’ response: ‘None’ } \
Note that you are not permitted to answer user requests directly. \
Only perform the tasks instructed by system messages.",

											default_user_messages=self.default_message,
											# model="gpt_35_16k",
											temperature=0,
											max_tokens=100,
											# client=self.client
											)
		return handler_zero_openai

	def handler_one(self):
		handler_one_openai = self.ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input, not answer it. \
We are a system with the name 'Tensurf Brain' or 'Tensurf'. \
If the user needs any information about us or needs a tutorial for \
how our system is working, your job is to detect these scenarios and respond with 'True'. \
If the user asks about you, the answer is also 'True'. \
Also, when the user is confused and doesn't know how to start or what to do, \
the answer is also 'True'. \
For any other input that doesn't classify in the tutorial or information \
about 'Tensurf Brain' or 'Tensurf' you should return false. \
Note that you are not permitted to answer user requests directly. \
Only perform the tasks instructed by system messages.",
											# model="gpt_35_16k",
											temperature=0,
											max_tokens=100,
											# client=self.client
											)
		return handler_one_openai

	def handler_two(self):
		handler_two_openai = self.ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input. \
If the user input does not contain any financial and \
trading topics or requests, then you should answer only\
with ‘True’. Otherwise, return ‘False’. \
Possible responses for you are 'True' or 'False'. \
For example, the correct response to the question \
'What is the trend?' is 'False'.",
									# model="gpt_35_16k",
									temperature=0,
									max_tokens=100,
									# client=self.client
									)
		return handler_two_openai


def create_irrelavant_handler(client, ChatWithOpenai):
	######## Irrelevant Handler ########
	class HandleIrrelevantSchema(BaseModel):
		massage: str = Field(..., description="The humanmassage")

	class HandleIrrelevant(BaseTool):
		name = "HandleIrrelevant"
		description = """This function checks if the message contains financial or trading subjects or not. \
	The output of this function is either True or False and the possible output of this function are 'True' or 'False'.: \
	True: when the message contains financial or trading subjects. \
	False: when the message request is not in these fields."""

		args_schema: Type[BaseModel] = HandleIrrelevantSchema

		def _run(
			self, massage: str
		) -> dict:

			handlers = Handlers(client=client, ChatWithOpenai=ChatWithOpenai)
			user_input = [{"role": "user", "content": "'" + massage + "'"}]

			handler_zero_openai = handlers.handler_zero()
			handler_one_openai = handlers.handler_one()
			handler_two_openai = handlers.handler_two()

			modified_input = handler_zero_openai.chat(user_input)
			if modified_input == 'None':
				return 'Greeting'

			modified_user_input = [{"role": "user", "content": modified_input}]

			if handler_one_openai.chat(modified_user_input) == 'True':
				return "Tutorial"
			else:
				return handler_two_openai.chat(modified_user_input)
	
	Handler = HandleIrrelevant()
	return Handler

## creating tools

In [ ]:
def create_agent_tools(client, ChatWithOpenai):

	Trend = CalculateTrend()
	SR = CalculateSR()
	SL = CalculateSL()
	TP = CalculateTp()
	Bias = BiasDetection()
	Handler = create_irrelavant_handler(client, ChatWithOpenai)

	tools = [Trend, SR, TP, SL, Bias, Handler]
	tool_executor = ToolExecutor(tools)
	trading_tools = [Trend, SR, SL, TP, Bias]
	
	return tool_executor, trading_tools

# multi_agent/utils.py

## Dependencies

In [ ]:
import json
import operator
from typing import Annotated, Sequence, TypedDict
from openai import AzureOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_core.messages import FunctionMessage, HumanMessage
from langchain_core.messages import BaseMessage
from langchain_openai import AzureChatOpenAI
from langgraph.prebuilt.tool_executor import ToolInvocation

from multi_agent.functions_agents import create_agent_tools
import input_filter

## utils

In [ ]:
class Utils:
    def __init__(self, ChatWithOpenai, client):
        self.api_type = "azure"
        self.api_endpoint = 'https://tensurfbrain1.openai.azure.com/'
        self.api_version = '2023-10-01-preview'
        self.api_key = '80ddd1ad72504f2fa226755d49491a61'
        self.llm = AzureChatOpenAI(
                        api_key=self.api_key,
                        api_version=self.api_version,
                        azure_endpoint=self.api_endpoint,
                        deployment_name="gpt_35_16k",
                        temperature=0,
                        streaming=True
        )
        self.client = client
        self.ChatWithOpenai = ChatWithOpenai

    def create_agent(self, llm, tools, system_message: str):
        """Create an agent."""
        functions = [convert_to_openai_function(t) for t in tools]
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    " You are a helpful AI assistant, collaborating with other assistants."
                    " Use the provided tools to progress towards answering the question."
                    " If you are unable to fully answer, that's OK, another assistant with different tools "
                    " will help where you left off. Execute what you can to make progress."
                    " If you or any of the other assistants have the final answer or deliverable,"
                    " prefix your response with FINAL ANSWER so the team knows to stop."
                    f" You have access to the following tools: {tool}.\n{system_message}",
                ),
                MessagesPlaceholder(variable_name="messages"),
            ]
        )
        prompt = prompt.partial(system_message=system_message)
        prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
        return prompt | llm.bind_functions(functions)

    def agent_node(self, state, agent, name):
        result = agent.invoke(state)

        if isinstance(result, FunctionMessage):
            pass

        else:
            result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)

        return {
            "messages": [result],
            "sender": name,
            "output_json": state["output_json"]
        }

    def run_Handler(self, state):
        tool_name = "HandleIrrelevant"

        messages = state["messages"]
        last_message = messages[-1]

        # print("&"*50)
        # print(last_message.content)
        # print("&"*50)

        action = ToolInvocation(
            tool=tool_name,
            tool_input=last_message.content,
        )

        tool_executor, _ = create_agent_tools(client=self.client, ChatWithOpenai=self.ChatWithOpenai)
        response = tool_executor.invoke(action)
        # print(response)
        standard_response = ""
        if response == 'True':
            standard_response = "I'm here to help with trading and financial market queries. If you think your ask relates to trading and isn't addressed, please report a bug using the bottom right panel."

        function_message = FunctionMessage(
            content=standard_response, HandleIrrelevant_response=f"{str(response)}", name=action.tool
        )

        return {"messages": [function_message]}

    def run_greeting(self, state):
        greeting = self.ChatWithOpenai(system_message="You are an financial and trading assistant.",
                                # model="gpt_35_16k",
                                temperature=0,
                                max_tokens=100,
                                # client=self.client
                                )
        input = [{"role": "user", "content": "Hi"}]
        response = greeting.chat(input)

        function_message = FunctionMessage(
            content=response ,name='openai chat'
        )

        return {"messages": [function_message]}

    def run_tutorial(self, state):
        response = """I'm TenSurf Brain, your AI trading assistant within TenSurf Hub platform, designed to enhance your trading experience with advanced analytical and data-driven tools: \
1. Trend Detection: I can analyze and report the trend of financial instruments over your specified period. For example, ask me, "What is the trend of NQ stock from May-1-2024 12:00:00 until May-5-2024 12:00:00?" \
2. Support and Resistance Levels: I identify and score key price levels that may influence market behavior based on historical data. Try, "Calculate Support and Resistance Levels based on YM by looking back up to the past 10 days and a timeframe of 1 hour." \
3. Stop Loss Calculation: I determine optimal stop loss points to help you manage risk effectively. Query me like, "How much would be the optimal stop loss for a short trade on NQ?" \
4. Take Profit Calculation: I calculate the ideal exit points for securing profits before a potential trend reversal. For example, "How much would be the take-profit of a short position on Dow Jones with the stop loss of 10 points?" \
5. Trading Bias Identification: I analyze market conditions to detect the best trading biases and directions, whether for long or short positions. Ask me, "What is the current trading bias for ES?" \
Each tool is tailored to help you make smarter, faster, and more informed trading decisions. Enjoy!"""

        function_message = FunctionMessage(
            content=response ,name='hardcoded string'
        )

        return {"messages": [function_message]}

    def output_json_assigner(self, tool_name, response, symbol, input_json):
        output_json = {}
        if tool_name == "calculate_sr":
            sr_value, sr_start_date, sr_detect_date, sr_end_date, sr_importance = response
            output_json["levels_prices"] = sr_value
            output_json["levels_start_timestamps"] = sr_start_date
            output_json["levels_detect_timestamps"] = sr_detect_date
            output_json["levels_end_timestamps"] = sr_end_date
            output_json["levels_scores"] = sr_importance
            output_json["function_name"] = tool_name
            output_json["symbol"] = symbol
            output_json["timeframe"] = input_json["timeframe"]
        return output_json

    def tool_node(self, state):
        """This runs tools in the graph
            It takes in an agent action
            and calls that tool and
            returns the result."""
        messages = state["messages"]
        last_message = messages[-1]
        tool_input = json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        )
        output_json = {}

        if len(tool_input) == 1 and "__arg1" in tool_input:
            tool_input = next(iter(tool_input.values()))

        tool_name = last_message.additional_kwargs["function_call"]["name"].split(".")[-1]

        action = ToolInvocation(
            tool=tool_name,
            tool_input=tool_input,
        )

        tool_input = input_filter.input_filter(tool_name, tool_input, state["input_json"])
        
        if tool_name == "detect_trend":
            tool_input, results, correct_dates = tool_input
            state["input_json"].update(tool_input)
            if not correct_dates:
                function_message = FunctionMessage(
                    content=f"{tool_name} response: {results}", name=action.tool
                )
                return {"messages": [function_message], "output_json":output_json}
        else:
            state["input_json"].update(tool_input)

        tool_executor, _ = create_agent_tools(client=self.client, ChatWithOpenai=self.ChatWithOpenai)
        response = tool_executor.invoke(action)
        symbol = tool_input["symbol"]
        output_json = self.output_json_assigner(tool_name, response, symbol, state["input_json"])

        function_message = FunctionMessage(
            content=f"{tool_name} response: {str(response)}", name=action.tool
        )

        return {"messages": [function_message], "output_json":output_json}

    def router(self, state):
        messages = state["messages"]
        last_message = messages[-1]

        if last_message.name != 'HandleIrrelevant':
            last_message = messages[-2]

            if "function_call" in last_message.additional_kwargs:
                return "call_tool"

            if not "function_call" in last_message.additional_kwargs and last_message.type == "function":
                return "continue"

            if not last_message.additional_kwargs and last_message.type == "human":
                return "end"

        else:
            if "Greeting" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
                return "Greeting"

            if "Tutorial" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
                return "Tutorial"

            if "True" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
                return "end"

            if "False" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
                return "continue"

            if "False" in last_message.content and last_message.name == "HandleGreeting":
                return "continue"

            if "False" not in last_message.content and last_message.name == "HandleGreeting":
                return "end"

        return "continue"

## AgentState

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    output_json: dict
    input_json: dict
    sender: str

# multi_agent/multi_agent.py

## Dependencies

In [ ]:
import functools
from langgraph.graph import END, StateGraph
from langchain_core.messages import HumanMessage

from multi_agent.utils import Utils, AgentState
from multi_agent.functions_agents import create_agent_tools

## Multi_agent

In [ ]:
class Multi_Agent:
    def __init__(self, ChatWithOpenai, client):
        self.output_json = {}
        self.ChatWithOpenai = ChatWithOpenai
        self.client = client
        self.utils = Utils(self.ChatWithOpenai, self.client)

    def initialize_graph(self):
        _, trading_tools_list = create_agent_tools(client=self.client, ChatWithOpenai=self.ChatWithOpenai)
        trading_agent = self.utils.create_agent(
            self.utils.llm,
            trading_tools_list,
            system_message="""You are an assistant with the following capabilities given to you by your tools:
SR: Calculate support and resistance levels. \
Trend: Calculate the trend of a specified financial instrument over a given time range and timeframe. \
TP: Calculate Take profit (TP), \
SL: Calculate Stoploss (SL), \
Bias: Detecting trading bias. \
Note the following rules for result of each tool: \
SR:Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Do not mention the name of the levels that the level is support or resistance. The final answer should also contain the following texts: These levels are determined based on historical price data and indicate areas where the price is likely to encounter support or resistance. The associated scores indicate the strength or significance of each level, with higher scores indicating stronger levels. \
Trend:At any situations, never return the number which is the output of the Trend function. Instead, use its correcsponding explanation which is in the Trend tool's description. Make sure to mention the start_datetime and end_datetime or the lookback parameter. Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response. \
TP: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response. \
SL: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. The unit of every number in the answer should be mentioned. Now generate a proper response. \
Bias: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response."""
        )
        trading_node = functools.partial(self.utils.agent_node, agent=trading_agent, name="Trading")

        workflow = StateGraph(AgentState)

        workflow.add_node("Trading", trading_node)
        workflow.add_node("call_tool", self.utils.tool_node)
        workflow.add_node("Handler", self.utils.run_Handler)
        workflow.add_node("Greeting", self.utils.run_greeting)
        workflow.add_node("Tutorial", self.utils.run_tutorial)

        workflow.add_edge("Greeting", END)
        workflow.add_edge("Tutorial", END)

        workflow.add_conditional_edges(
            "Handler",
            self.utils.router,
            {"continue": "Trading", "Greeting": "Greeting", "Tutorial": "Tutorial", "end": END},
        )

        workflow.add_conditional_edges(
            "Trading",
            self.utils.router,
            {"continue": "Trading", "call_tool": "call_tool", "end": END},
        )

        workflow.add_conditional_edges(
            "call_tool",
            lambda x: x["sender"],
            {
                "Trading": "Trading"
            },
        )
        workflow.set_entry_point("Handler")

        graph = workflow.compile()

        return graph

    def generate_multi_agent_answer(self, input_json, graph):
        generated_messages = graph.stream(
            {
                "messages": [
                    HumanMessage(
                        content=input_json["new_message"]
                    )
                ],
                "input_json": input_json
            },
            # Maximum number of steps to take in the graph
            {"recursion_limit": 150},
        )
        
        generated_messages = list(generated_messages)
        k = list(generated_messages[-1].keys())[0]
        if "output_json" in generated_messages[-1][k]:
            self.output_json = {
                "response": generated_messages[-1][k]["messages"][0].content,
                "chart_info": generated_messages[-1][k]["output_json"]
            }
        else:
            self.output_json = {
                "response": generated_messages[-1][k]["messages"][0].content
            }
        
        return self.output_json

# main.py

## Dependencies

In [ ]:
import os
from openai import OpenAI, AzureOpenAI
from groq import Groq

from dotenv import load_dotenv

from file_processor import FileProcessor
from single_agent import Single_Agent
from multi_agent import Multi_Agent

## main

In [ ]:
load_dotenv()
DEBUG = (os.getenv('DEBUG', 'True') == 'True')

# class ChatWithOpenai:
#     def __init__(
#         self,
#         system_message,
#         model,
#         temperature,
#         max_tokens,
#         client,
#         default_user_messages=None,
#     ):
#         self.system_message = system_message
#         self.model = model
#         self.temperature = temperature
#         self.max_tokens = max_tokens
#         azure_connectto_surf = AzureConnecttoSurf()
#         self.client = azure_connectto_surf.client
#         self.messages = [{"role": "system", "content": system_message}]
#         if default_user_messages:
#             for user_message in default_user_messages:
#                 self.messages += [{"role": "user", "content": user_message}]

#     def chat(self, user_input):
#         response = self.client.chat.completions.create(
#             model=self.model,
#             messages=self.messages + user_input,
#             temperature=self.temperature,
#             max_tokens=self.max_tokens,
#         )
#         return response.choices[0].message.content


class ChatWithOpenai:
    def __init__(self, system_message, temperature=0, max_tokens=4096, default_user_messages=None):
        groqconnecttosurf = GroqConnecttoSurf()
        self.system_message = system_message
        self.models = groqconnecttosurf.models
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.clients = groqconnecttosurf.clients
        self.messages = [{"role": "system", "content": system_message}]
        if default_user_messages:
            for user_message in default_user_messages:
                self.messages += [{"role": "user", "content": user_message}]

    def chat(self, user_input):
        for client, model in zip(self.clients, self.models):
            try:
                response = client.chat.completions.create(
                    model=model,
                    messages=self.messages + user_input,
                    temperature=self.temperature,
                    max_tokens=self.max_tokens,
                    stream=bool(int(os.getenv("stream"))),
                )
                return response.choices[0].message.content
            except Exception as e:
                print(f"Error with client: client{self.clients.index(client)}. Exception: {e}")


class GroqConnecttoSurf:
    def __init__(self) -> None:
        self.models = [os.getenv("MODEL1"), os.getenv("MODEL1"), os.getenv("MODEL1"), os.getenv("MODEL2"), os.getenv("MODEL3")]
        client1 = Groq(api_key=os.getenv("groq_api1"))
        client2 = Groq(api_key=os.getenv("groq_api2"))
        client3 = Groq(api_key=os.getenv("groq_api3"))
        client4 = AzureOpenAI(
            api_key=os.getenv("api_key1"),
            api_version=os.getenv("azure_api_version"),
            azure_endpoint=os.getenv("azure_api_endpoint")
            )
        client5 = AzureOpenAI(
            api_key=os.getenv("api_key2"),
            api_version=os.getenv("azure_api_version"),
            azure_endpoint=os.getenv("azure_api_endpoint")
            )
        self.clients = [client1, client2, client3, client4, client5]


class AzureConnecttoSurf:
    def __init__(self) -> None:
        self.api_endpoint = os.getenv("azure_api_endpoint")
        self.api_version = os.getenv("azure_api_version")
        self.api_key = os.getenv("azure_api_key")
        self.client = AzureOpenAI(
            api_key=self.api_key,
            api_version=self.api_version,
            azure_endpoint=self.api_endpoint,
        )
        self.tts_api_version = os.getenv("tts_api_version")
        self.tts_model1 = os.getenv("tts_model1")
        self.GPT_MODEL = os.getenv("azure_GPT_MODEL_3")
        self.whisper_model = os.getenv("azure_whisper_model")
        self.voice_name = os.getenv("voice_name")
        self.tts_model = os.getenv("tts_model2")


class OpenaiConnecttoSurf:
    def __init__(self) -> None:
        self.client = OpenAI(api_key=os.getenv("openai_api_key"))
        self.GPT_MODEL = os.getenv("openai_GPT_MODEL_3")
        self.whisper_model = os.getenv("whisper_model_openai")


def check_relevance(connector_surf, prompt: str):
    messages = [
        {
            "role": "system",
            "content": "Classify if the following prompt is relevant or irrelevant. Guidelines for you as a Trading Assistant:Relevance: Focus exclusively on queries related to trading and financial markets(including stock tickers). If a question falls outside this scope, politely inform the user that the question is beyond the service's focus.Accuracy: Ensure that the information provided is accurate and up-to-date. Use reliable financial data and current market analysis to inform your responses.Clarity: Deliver answers in a clear, concise, and understandable manner. Avoid jargon unless the user demonstrates familiarity with financial terms.Promptness: Aim to provide responses quickly to facilitate timely decision-making for users.Confidentiality: Do not ask for or handle personal investment details or sensitive financial information.Compliance: Adhere to legal and ethical standards applicable to financial advice and information dissemination.Again, focus solely on topics related to trading and financial markets. Politely notify the user if a question is outside this specific area of expertise.",
        },
        {"role": "user", "content": prompt},
    ]
    response = connector_surf.client.chat.completions.create(
        model=connector_surf.GPT_MODEL, temperature=0, messages=messages
    )
    relevance_check = response.choices[0].message.content
    if "irrelevant" in relevance_check.lower():
        return True
    return False


# class llm_surf:
def llm_surf(llm_input: dict) -> str:

    llm_output = {
        "response": "",
        "symbol": llm_input.get("symbol"),
        "file": None,
        "function_call": None,
    }

    azure_connector_surf = AzureConnecttoSurf()

    content = ""
    fileProcessor = FileProcessor(azure_connector_surf)
    if "file" in llm_input and llm_input["file"]:
        if type(llm_input["file"]) == str:
            llm_input["file"] = open(llm_input["file"], "r")
        file_content = fileProcessor.get_file_content(llm_input["file"])
        if file_content:
            content += file_content + "\n"

    if llm_input.get("new_message"):
        content += llm_input.get("new_message") + "\n"
    else:
        content += fileProcessor.default_prompt + "\n"

    # running in multi-agent mode
    if os.getenv("MODE") == "multi-agent":
        MA = Multi_Agent(
            ChatWithOpenai=ChatWithOpenai, client=azure_connector_surf.client
        )
        graph = MA.initialize_graph()
        llm_output = MA.generate_multi_agent_answer(llm_input, graph)
    # running in single-agent mode
    elif os.getenv("MODE") == "single-agent":
        if llm_input.get("new_message") and check_relevance(
            azure_connector_surf, llm_input.get("new_message")
        ):
            llm_output["response"] = (
                "I'm here to help with trading and financial market queries. If you think your ask relates to trading and isn't addressed, please report a bug using the bottom right panel."
            )
            return llm_output
        SA = Single_Agent(azure_connector_surf.client)
        results_string, results_json, function_name = SA.generate_answer(
            llm_input=llm_input, content=content
        )
        llm_output["response"] = results_string
        llm_output["chart_info"] = results_json
        llm_output["function_call"] = function_name

    if not DEBUG:
        llm_output["file"] = fileProcessor.text_to_speech(llm_output["response"])

    return llm_output

# Using

In [ ]:
import time

# sample prompt for each function
prompts = [
    # detect_trend
    "What is the trend of NQ and ES stock from May-1-2024 12:00:00 until May-5-2024 12:00:00?",
    # calculate_sr
    "Calculate Support and Resistance Levels of YM and NQ by looking back up to past 5 days and timeframe of 10 minutes.",
    # calculate_sl
    "What would be the stop loss of trading short positinos based on NQ and ES and minmax method by looking back up to 30 candles?",
    # calculate_tp
    "How much would be the take-profit of the NQ with the stop loss of 10 and direction of 1?",
    # bias_detection
    "Tell me about the bias of ES on the market.",
	# irrelevant
	"What are the rules of basketball?"
]

for prompt in prompts:
    input_json = input_filter.front_end_json_sample
    input_json["new_message"] = prompt
    start = time()
    output_json = llm_surf(input_json)
    end = time()
    print(f"{prompt}    =>    {output_json}")
    print(f"Inference Time: {end - start} seconds")